In [31]:
#install required packages
!pip install groq pandas scikit-learn tensorflow streamlit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.8 MB/s eta 0:00:00


In [33]:
# ========== 1. IMPORTS ==========
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import pickle
from groq import Groq
import datetime

In [34]:
GROQ_API_KEY = "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"  # 🔁 Replace with your actual key
client = Groq(api_key=GROQ_API_KEY)

In [35]:
# ========== 3. Symptom Columns ==========
symptom_columns = [
    "fever", "cough", "shortness of breath", "fatigue", "chest pain", "headache",
    "vomiting", "diarrhea", "joint pain", "muscle pain", "anxiety", "nausea",
    "sore throat", "rash", "sweating", "loss of appetite", "insomnia", "runny nose",
    "dizziness", "abdominal pain", "weight loss", "congestion", "blurred vision",
    "palpitations", "dry mouth", "tingling", "loss of smell", "loss of taste",
    "skin irritation", "cold hands and feet", "irritability", "bloating",
    "trouble concentrating", "back pain", "eye redness", "ear pain", "lightheadedness",
    "increased thirst", "excessive urination", "difficulty swallowing",
    "itchy eyes", "night sweats", "hoarseness", "neck stiffness", "swollen glands",
    "frequent urination", "excessive hunger", "sensitivity to light"
]

In [36]:
# ========== 4. GROQ Functions ==========
def extract_symptoms_with_groq(user_input):
    prompt = f"""
You are a medical assistant.
Extract symptoms from the user's message below. Only return symptoms from this list:
{', '.join(symptom_columns)}

User: {user_input}
List the symptoms as an array in Python format, e.g. ["cough", "fever"]
"""
    res = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": "You are a medical assistant that only returns valid symptoms."},
            {"role": "user", "content": prompt}
        ]
    )
    try:
        return eval(res.choices[0].message.content.strip())
    except:
        return []

def get_diagnosis_and_treatment(user_input):
    prompt = f"""
You are a smart AI medical assistant. A user described their symptoms like this:
"{user_input}"

1. Identify likely disease from the description.
2. Provide a friendly 2-3 line treatment/advice.
Do not suggest to consult a doctor unless it's critical. Be concise and helpful.
Format:
Disease: <name>
Advice: <short paragraph>
"""
    res = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return res.choices[0].message.content.strip()


In [37]:
# ========== 5. Session Log ==========
def log_session(user_input, detected_symptoms, result_text):
    data = {
        "timestamp": [datetime.datetime.now().isoformat()],
        "user_input": [user_input],
        "symptoms": ["; ".join(detected_symptoms)],
        "result": [result_text]
    }
    df = pd.DataFrame(data)
    df.to_csv("session_logs.csv", mode="a", header=False, index=False)


In [46]:
# ========== 6. MAIN APP ==========
print("📝 Please describe your symptoms in one or two sentences:")
user_input = input("Your description: ")

# Step 1: Extract symptoms
found_symptoms = extract_symptoms_with_groq(user_input)
print("🔍 Detected Symptoms:", found_symptoms)

# Step 2: Diagnose and treat using LLM only
result = get_diagnosis_and_treatment(user_input)
print("\n🤖 Diagnosis and Advice:\n", result)

# Step 3: Save session
log_session(user_input, found_symptoms, result)
print("🗂️ Session saved to log file.")


📝 Please describe your symptoms in one or two sentences:
Your description: synuse
🔍 Detected Symptoms: []

🤖 Diagnosis and Advice:
 Based on the symptom "synuse", I'm interpreting it as "sinus".

Disease: Sinusitis

Advice: You're likely experiencing sinusitis, an inflammation or infection of the sinus cavities. Try taking a warm compress over your face to loosen up mucus and ease pressure. You can also use a humidifier to add moisture to the air and reduce congestion. Over-the-counter pain relievers like acetaminophen or ibuprofen can also help alleviate symptoms.
🗂️ Session saved to log file.
